
# Week‑1 Samples Assembler (DocILE, CORD‑v2, SROIE)

Этот ноутбук собирает **небольшую выборку образцов** (samples) для R&D на Неделю 1 из **локально доступных** датасетов:
- **DocILE** (инвойсы/заказы)
- **CORD‑v2** (розничные чеки)
- **SROIE** (чеки)

> ⚠️ **Важно:** ноутбук **ничего не скачивает**. Он предполагает, что у вас уже есть локальные копии датасетов. 
> Проверьте условия лицензирования каждого набора и используйте только допустимым способом.



## Что делает ноутбук
1. Находит входные файлы (PDF/JPG/PNG) и ищет **аннотации** (JSON/TXT) поблизости по эвристикам.
2. Сэмплирует фиксированное число документов для каждого датасета.
3. Копирует выбранные примеры в структуру `samples/…` и создаёт **единый манифест** `samples/_manifests/manifest.csv`.
4. (Опционально) Вы можете адаптировать функции обнаружения аннотаций под вашу структуру.

## Структура результата
```
samples/
  invoices/docile/           # выбранные примеры DocILE (инвойсы)
    <files> + labels/
  receipts/cord/             # выбранные примеры CORD-v2 (чеки)
    <files> + labels/
  receipts/sroie/            # выбранные примеры SROIE (чеки)
    <files> + labels/
  _manifests/manifest.csv    # единый манифест
README.samples.md
```



## 1) Конфигурация

Укажите пути к локальным копиям датасетов и желаемое количество документов. 
Вы также можете передать параметры через переменные окружения (например, в Docker/RunPod).


In [ ]:

from pathlib import Path
import os

# === УКАЖИТЕ СВОИ ПУТИ (или оставьте None, если набора нет) ===
DOCILE_ROOT = os.getenv("DOCILE_ROOT", None) or None      # пример: "/data/docile"
CORD_ROOT   = os.getenv("CORD_ROOT", None) or None        # пример: "/data/cord-v2"
SROIE_ROOT  = os.getenv("SROIE_ROOT", None) or None       # пример: "/data/sroie"

OUT_DIR     = Path(os.getenv("SAMPLES_OUT", "samples"))
N_DOCILE    = int(os.getenv("N_DOCILE", 12))
N_CORD      = int(os.getenv("N_CORD", 12))
N_SROIE     = int(os.getenv("N_SROIE", 6))
SEED        = int(os.getenv("SAMPLES_SEED", 42))

DOCILE_ROOT, CORD_ROOT, SROIE_ROOT, OUT_DIR, N_DOCILE, N_CORD, N_SROIE, SEED



## 2) Импорты и базовые типы
Только стандартная библиотека — чтобы ноутбук легко запускался в разных окружениях.


In [ ]:

import csv
import hashlib
import random
import re
import shutil
from dataclasses import dataclass
from typing import Optional, List, Dict, Tuple

IMG_EXTS = {".jpg", ".jpeg", ".png", ".tif", ".tiff"}
PDF_EXTS = {".pdf"}
LABEL_EXTS = {".json", ".txt"}

@dataclass
class FoundItem:
    src_path: Path
    label_path: Optional[Path]
    split: str  # 'train' | 'val' | 'test' | 'unknown'



## 3) Вспомогательные функции
- Хэширование (sha256)
- Определение сплита по пути
- Поиск кандидатов аннотаций рядом с файлом


In [ ]:

def sha256sum(p: Path) -> str:
    h = hashlib.sha256()
    with p.open('rb') as f:
        for chunk in iter(lambda: f.read(1024 * 1024), b''):
            h.update(chunk)
    return h.hexdigest()

def guess_split(path: Path) -> str:
    s = path.as_posix().lower()
    for key in ("train", "training"):
        if f"/{key}/" in s or s.endswith(f"/{key}"):
            return "train"
    for key in ("val", "valid", "validation"):
        if f"/{key}/" in s or s.endswith(f"/{key}"):
            return "val"
    for key in ("test", "testing"):
        if f"/{key}/" in s or s.endswith(f"/{key}"):
            return "test"
    return "unknown"

def numeric_core(stem: str) -> str:
    m = re.search(r"(\d+)", stem)
    return m.group(1) if m else stem

def find_label_candidates(root: Path, target_stem: str) -> List[Path]:
    """Ищем файлы разметки поблизости по exact stem и по numeric core."""
    core = numeric_core(target_stem)
    candidates: List[Path] = []
    up = root
    for _ in range(2):  # два уровня вверх (и вниз через rglob)
        for ext in LABEL_EXTS:
            candidates += list(up.rglob(f"{target_stem}{ext}"))
            candidates += list(up.rglob(f"*{core}{ext}"))
        up = up.parent
    uniq = []
    seen = set()
    for c in candidates:
        if c.exists():
            key = c.resolve()
            if key not in seen:
                uniq.append(c)
                seen.add(key)
    return uniq



## 4) Обнаружение файлов датасетов
Эвристики разные для каждого набора.


In [ ]:

def discover_docile(docile_root: Path) -> List[FoundItem]:
    items: List[FoundItem] = []
    for p in docile_root.rglob("*"):
        if p.is_file() and p.suffix.lower() in (IMG_EXTS | PDF_EXTS):
            labels = find_label_candidates(p.parent, p.stem)
            label = next((x for x in labels if x.suffix.lower() == ".json"), None)
            items.append(FoundItem(src_path=p, label_path=label, split=guess_split(p)))
    return items

def discover_cord(cord_root: Path) -> List[FoundItem]:
    """CORD-v2: обычно изображения в {split}/image, метки — в {split}/label."""
    items: List[FoundItem] = []
    for p in cord_root.rglob("*"):
        if p.is_file() and p.suffix.lower() in IMG_EXTS:
            split = guess_split(p)
            label = None
            maybe_split_dir = p.parent.parent if p.parent else cord_root
            label_dir = maybe_split_dir / "label"
            if label_dir.exists():
                cand = label_dir / f"{p.stem}.json"
                if cand.exists():
                    label = cand
            if label is None:
                labels = find_label_candidates(p.parent, p.stem)
                label = next((x for x in labels if x.suffix.lower() == ".json"), None)
            items.append(FoundItem(src_path=p, label_path=label, split=split))
    return items

def discover_sroie(sroie_root: Path) -> List[FoundItem]:
    items: List[FoundItem] = []
    for p in sroie_root.rglob("*"):
        if p.is_file() and p.suffix.lower() in IMG_EXTS:
            split = guess_split(p)
            labels = find_label_candidates(p.parent, p.stem)
            label = next((x for x in labels if x.suffix.lower() in (".txt", ".json")), None)
            items.append(FoundItem(src_path=p, label_path=label, split=split))
    return items



## 5) Сэмплирование
Простая случайная выборка фиксированного размера.


In [ ]:

def pick_samples(items: List[FoundItem], n: int, seed: int) -> List[FoundItem]:
    items = list(items)
    rnd = random.Random(seed)
    rnd.shuffle(items)
    return items[:n] if n > 0 else []



## 6) Копирование файлов и формирование манифеста
Копируем файлы и (если найдены) метки в целевую иерархию, считаем `sha256`, пишем `manifest.csv`.


In [ ]:

def copy_item(item: FoundItem, out_dir: Path) -> Tuple[str, Optional[str], str]:
    out_dir.mkdir(parents=True, exist_ok=True)
    dst = out_dir / item.src_path.name
    if not dst.exists():
        shutil.copy2(item.src_path, dst)
    label_name = None
    if item.label_path and item.label_path.exists():
        labels_dir = out_dir / "labels"
        labels_dir.mkdir(parents=True, exist_ok=True)
        ldst = labels_dir / item.label_path.name
        if not ldst.exists():
            shutil.copy2(item.label_path, ldst)
        label_name = ldst.name
    return dst.name, label_name, sha256sum(dst)

def write_manifest(rows: List[Dict[str, str]], manifest_path: Path) -> None:
    manifest_path.parent.mkdir(parents=True, exist_ok=True)
    with manifest_path.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=[
                "dataset","doc_type","split","subset_dir","filename","label_filename",
                "has_gt","sha256","src_path","src_label_path"]
        )
        writer.writeheader()
        for r in rows:
            writer.writerow(r)



## 7) Обнаружение доступных документов
Запускаем проход по указанным корневым папкам датасетов.


In [ ]:

docile_items = discover_docile(Path(DOCILE_ROOT)) if DOCILE_ROOT else []
cord_items   = discover_cord(Path(CORD_ROOT))   if CORD_ROOT else []
sroie_items  = discover_sroie(Path(SROIE_ROOT)) if SROIE_ROOT else []

print(f"[docile] found: {len(docile_items)}" if DOCILE_ROOT else "[docile] skipped")
print(f"[cord]   found: {len(cord_items)}" if CORD_ROOT else "[cord]   skipped")
print(f"[sroie]  found: {len(sroie_items)}" if SROIE_ROOT else "[sroie]  skipped")



## 8) Сэмплирование и копирование
Выбираем нужное количество документов, копируем их в `samples/…`, собираем строки манифеста.


In [ ]:

rows: List[Dict[str, str]] = []

# DocILE (invoices)
if DOCILE_ROOT and docile_items:
    picked = pick_samples(docile_items, N_DOCILE, SEED)
    out_dir = OUT_DIR / "invoices" / "docile"
    for it in picked:
        fname, lfname, sha = copy_item(it, out_dir)
        rows.append({
            "dataset": "docile",
            "doc_type": "invoice",
            "split": it.split,
            "subset_dir": str(out_dir.relative_to(OUT_DIR)),
            "filename": fname,
            "label_filename": lfname or "",
            "has_gt": "1" if lfname else "0",
            "sha256": sha,
            "src_path": str(it.src_path.resolve()),
            "src_label_path": str(it.label_path.resolve()) if it.label_path else "",
        })
    print(f"[docile] picked: {len(picked)} -> {out_dir}")

# CORD-v2 (receipts)
if CORD_ROOT and cord_items:
    picked = pick_samples(cord_items, N_CORD, SEED)
    out_dir = OUT_DIR / "receipts" / "cord"
    for it in picked:
        fname, lfname, sha = copy_item(it, out_dir)
        rows.append({
            "dataset": "cord-v2",
            "doc_type": "receipt",
            "split": it.split,
            "subset_dir": str(out_dir.relative_to(OUT_DIR)),
            "filename": fname,
            "label_filename": lfname or "",
            "has_gt": "1" if lfname else "0",
            "sha256": sha,
            "src_path": str(it.src_path.resolve()),
            "src_label_path": str(it.label_path.resolve()) if it.label_path else "",
        })
    print(f"[cord-v2] picked: {len(picked)} -> {out_dir}")

# SROIE (receipts)
if SROIE_ROOT and sroie_items:
    picked = pick_samples(sroie_items, N_SROIE, SEED)
    out_dir = OUT_DIR / "receipts" / "sroie"
    for it in picked:
        fname, lfname, sha = copy_item(it, out_dir)
        rows.append({
            "dataset": "sroie",
            "doc_type": "receipt",
            "split": it.split,
            "subset_dir": str(out_dir.relative_to(OUT_DIR)),
            "filename": fname,
            "label_filename": lfname or "",
            "has_gt": "1" if lfname else "0",
            "sha256": sha,
            "src_path": str(it.src_path.resolve()),
            "src_label_path": str(it.label_path.resolve()) if it.label_path else "",
        })
    print(f"[sroie] picked: {len(picked)} -> {out_dir}")

print(f"[total rows] {len(rows)}")



## 9) Запись манифеста и README


In [ ]:

manifest_path = OUT_DIR / "_manifests" / "manifest.csv"
write_manifest(rows, manifest_path)

readme = OUT_DIR / "README.samples.md"
readme.write_text(
    "# Week-1 Samples\n\n"
    "- This folder contains reduced samples for R&D (Week 1).\n"
    "- Manifest: `_manifests/manifest.csv`.\n"
    "- Subsets:\n"
    "  - `invoices/docile/` — DocILE samples (invoices)\n"
    "  - `receipts/cord/` — CORD-v2 samples (retail receipts)\n"
    "  - `receipts/sroie/` — SROIE samples (receipts)\n\n"
    "Notes:\n"
    "- Ground-truth labels are copied to `labels/` when found via heuristics.\n"
    "- If labels are absent in your local copy, `has_gt=0` will be set.\n"
    , encoding="utf-8"
)
manifest_path, readme



## 10) Предпросмотр манифеста (опционально)
Если установлен `pandas`, выведем интерактивную таблицу; иначе покажем первые строки.


In [ ]:

from pathlib import Path
manifest_path = OUT_DIR / "_manifests" / "manifest.csv"
if manifest_path.exists():
    try:
        import pandas as pd
        df = pd.read_csv(manifest_path)
        # Отображение для пользователя (в веб‑интерфейсе ChatGPT будет табличка)
        from caas_jupyter_tools import display_dataframe_to_user
        display_dataframe_to_user("Week-1 Manifest", df)
    except Exception as e:
        print("pandas не установлен или отображение недоступно. Покажем первые строки файла:\n")
        print("\n".join(manifest_path.read_text(encoding="utf-8").splitlines()[:10]))
else:
    print("Manifest not found:", manifest_path)



## 11) Хуки для интеграции с вашим репозиторием
Если у вас есть адаптеры в `lib/datasets/adapters/*`, вы можете:
- нормализовать метки под вашу Pydantic‑схему (`Invoice/Receipt`),
- складывать эталоны в `labels/*.gt.json` для `scripts/eval.py`.

Добавьте соответствующий код в отдельную ячейку ниже.


In [ ]:

# Пример (псевдокод):
# from lib.datasets.adapters.docile import normalize_docile_label
# for lbl in (OUT_DIR / "invoices" / "docile" / "labels").glob("*.json"):
#     tgt = lbl.with_suffix(".gt.json")
#     data = normalize_docile_label(lbl.read_text(encoding="utf-8"))
#     tgt.write_text(json.dumps(data, ensure_ascii=False, indent=2), encoding="utf-8")
